In [ ]:
!pip -q install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 24.3 MB/s eta 0:00:00


In [ ]:
!pip -q install transformers datasets sentencepiece


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, RagTokenizer, RagRetriever, RagSequenceForGeneration
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch


In [ ]:
# Load Hugging Face's pre-trained RAG model (for retrieval) and MBERT (for QA)
rag_model_name = "facebook/rag-sequence-nq"
qa_model_name = "deepset/xlm-roberta-large-squad2"  # MBERT based for multilingual QA

In [ ]:
# Load RAG Tokenizer and Model
rag_tokenizer = RagTokenizer.from_pretrained(rag_model_name)
rag_retriever = RagRetriever.from_pretrained(rag_model_name, index_name="exact", use_dummy_dataset=True)
rag_model = RagSequenceForGeneration.from_pretrained(rag_model_name, retriever=rag_retriever)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The to

wiki_dpr.py:   0%|          | 0.00/8.63k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

The repository for wiki_dpr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wiki_dpr.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train-00000-of-00001.parquet:   0%|          | 0.00/40.8M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/10 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/rag-sequence-nq were not used when initializing RagSequenceForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagSequenceForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagSequenceForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Load the QA pipeline with MBERT
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
qa_pipeline = pipeline('question-answering', model=qa_model, tokenizer=qa_tokenizer)

tokenizer_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Tamil context (this will be provided by the user)
tamil_context = """
தமிழ்நாடு இந்தியாவின் தெற்கே அமைந்துள்ளது. இதன் தலைநகரம் சென்னை. தமிழ் மொழி இதன் அதிகாரப்பூர்வ மொழியாகும்.
தமிழ்நாடு தென்னிந்தியாவில் அமைந்துள்ள மிகவும் புகழ்பெற்ற மாநிலமாகும். இதன் பண்பாட்டில் பெரும் சிறப்புமிக்கது மற்றும்
பொதுவாகவே ஆன்மிகம் மற்றும் கலாசார பாரம்பரியத்தில் அதிக கவனம் செலுத்துகிறது.
"""


# Example English question
english_question = "What is the official language of Tamil Nadu?"

In [ ]:
# Step 1: Use RAG to retrieve the most relevant Tamil context for the question
def retrieve_context(question, context):
    # Tokenize the question for RAG retrieval
    inputs = rag_tokenizer(question, return_tensors="pt")
    retrieved_docs = rag_retriever(inputs['input_ids'], inputs['attention_mask'], return_tensors="pt")

    # Generate with RAG to get the relevant context
    generated_ids = rag_model.generate(input_ids=inputs['input_ids'], context_input_ids=retrieved_docs['context_input_ids'])

    # Decode the generated text (the most relevant context)
    relevant_context = rag_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return relevant_context


In [ ]:

# Step 2: Use QA pipeline to answer the question using the retrieved context
def answer_question_with_mbert(question, context):
    input_dict = {
        'question': question,
        'context': context
    }
    answer = qa_pipeline(input_dict)
    return answer['answer']



In [1]:

# Retrieve the relevant Tamil context for the English question
retrieved_context = retrieve_context(english_question, tamil_context)

# Answer the question using the retrieved context
answer = answer_question_with_mbert(english_question, retrieved_context)


In [ ]:

# Output
print(f"Question: {english_question}")
print(f"Context: {retrieved_context}")
print(f"Answer: {answer}")